### requirements for the following codings


In [4]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

^C
Traceback (most recent call last):
  File "/Users/zhenjiaodu/opt/anaconda3/envs/unidl/bin/pip", line 4, in <module>
    import re
  File "/Users/zhenjiaodu/opt/anaconda3/envs/unidl/lib/python3.9/re.py", line 125, in <module>
    import sre_compile
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 941, in get_code
  File "<frozen importlib._bootstrap_external>", line 1039, in get_data
KeyboardInterrupt
  Using cached torch-2.2.2-cp39-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached netwo

### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [ ]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings (main dataset)

In [1]:
import numpy as np
import pandas as pd

In [ ]:
# training dataset loading
dataset = pd.read_excel('bitter_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('bitter_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

In [ ]:
# test dataset loading
dataset = pd.read_excel('bitter_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('bitter_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')
# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [2]:
# training dataset loading
dataset = pd.read_excel('bitter_train.xlsx',na_filter = False) # take care the NA sequence problem
# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model
# test dataset loading
dataset = pd.read_excel('bitter_test.xlsx',na_filter = False) # take care the NA sequence problem
# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model
# assign the dataset 
X_train_data_name = 'bitter_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'bitter_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# re-divide dataset
from sklearn.model_selection import train_test_split
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123,shuffle=True, stratify=y_train)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train_sub)
X_train_sub = scaler.transform(X_train_sub) # normalize X to 0-1 range
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
# check the dimension of the dataset before model development
print(X_train_sub.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train_sub.shape)
print(y_val.shape)
print(y_test.shape)

# Save the scaler to a file
import joblib
joblib.dump(scaler, 'minmax_scaler.pkl')
# # Load the scaler from the file
# loaded_scaler = joblib.load('minmax_scaler.pkl')

(409, 320)
(103, 320)
(128, 320)
(409,)
(103,)
(128,)


['minmax_scaler.pkl']

### Model architecture & evaluation

In [3]:
from keras.layers import Input,Dense, Activation, BatchNormalization, Flatten, Conv1D,Dropout, MaxPooling1D
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
import keras
from keras import backend as K
def ESM_CNN(X_train, y_train, X_valid, y_valid):
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(32,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)
  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)
  mc = ModelCheckpoint('best_model.keras',  monitor='val_accuracy', mode='max', verbose=1, save_best_only=True, save_weights_only=False)
  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop, mc]
  model_history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                            epochs=200, callbacks=callbacks_list,batch_size = 8, verbose=2)
  return model, model_history

2024-11-19 15:18:02.810322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, roc_auc_score
import numpy as np
import math
def model_evaluation(model_name, X_test, y_test):
    # Load the saved model
    saved_model = load_model(model_name)
    # Predict class probabilities
    predicted_protability = saved_model.predict(X_test, batch_size=1) 
    predicted_class = np.argmax(predicted_protability, axis=1) # operating horizontally /// row-wise
    # True labels
    y_true = y_test
    # Calculate confusion matrix components
    TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel()
    # Calculate evaluation metrics
    ACC = (TP + TN) / (TP + TN + FP + FN)
    Sn = TP / (TP + FN)
    Sp = TN / (TN + FP)
    MCC = (TP * TN - FP * FN) / (math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))) 
    BACC = 0.5 * Sn + 0.5 * Sp
    AUC = roc_auc_score(y_true, predicted_protability[:, 1])
    # Print results
    print("Accuracy:", ACC)
    print("Balanced Accuracy:", BACC)
    print("Sensitivity (Recall):", Sn)
    print("Specificity:", Sp)
    print("MCC:", MCC)
    print("AUC:", AUC)
    return ACC, BACC, Sn, Sp, MCC, AUC

### model evaluation in test dataset

In [ ]:
model, model_history = ESM_CNN(X_train_sub, y_train_sub, X_val , y_val)

In [5]:
import tensorflow as tf
tf.__version__

'2.12.1'

In [8]:
model_evaluation('best_model.keras',X_test,y_test)


128/128 [==============================] - 1s 2ms/step
Accuracy: 0.9453125
Balanced Accuracy: 0.9453125
Sensitivity (Recall): 0.96875
Specificity: 0.921875
MCC: 0.8916050852754615
AUC: 0.974853515625


(0.9453125, 0.9453125, 0.96875, 0.921875, 0.8916050852754615, 0.974853515625)